In [54]:
# 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
# http://www.menupan.com/restaurant/bestrest/bestrest.asp?pt=rt&areacode=ss204
# 전체 내용 데이터를 크롤링하기
# parser 종류 : html.parser => html5lib
url = "http://www.menupan.com/restaurant/bestrest/bestrest.asp?pt=rt&areacode=ss204"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html5lib') # html.parser, html5lib, lxml, ...
print(soup)


In [73]:
# 순위, 상호, 업종, 주소, url에 대한 내용을 출력

# select() vs select_one()
# select() : 매칭되는 여러 값들을 리스트에 담음
# select_one() : 매칭되는 값 하나만 가져와서 => 객체

# select관련 => 하나의 열에 여러 선택자가 필요한 경우
#   => .000 or .XXX => (or : ',' (쉼표))

# 속성값 구해야 할때 : [ ]를 사용!!

s = 'body > div > div.container > div.bestRest > div.ranking > div.rankingList > ul'
ulTag = soup.select_one(s)
# print(ulTag)

for li in ulTag.select('li'):
  print('순위 : ', li.select_one('.numTop, .rankNum').string)
  print('상호 : ', li.select_one('.restName').string)
  print('업종 : ', li.select_one('.listType').string)
  print('주소 : ', li.select_one('.listArea').string)
  # print('url : ', li.select_one('.listPhoto a')['href'])
  print('url : ', li.select_one('.restName a')['href'])
  print('==' * 30)


순위 :  1
상호 :  사시미야센
업종 :  일식
주소 :  양천구 목동
url :  /restaurant/onepage.asp?acode=J103874
순위 :  2
상호 :  명가들깨칼국수
업종 :  한식
주소 :  양천구 신정1동
url :  /restaurant/onepage.asp?acode=H119942
순위 :  3
상호 :  홍대돈부리 목동점
업종 :  일식
주소 :  양천구 목1동
url :  /restaurant/onepage.asp?acode=J104119
순위 :  4
상호 :  팡코 목동점
업종 :  베이커리
주소 :  양천구 목4동
url :  /restaurant/onepage.asp?acode=G101623
순위 :  5
상호 :  고성막국수
업종 :  한식
주소 :  강서구 방화동
url :  /restaurant/onepage.asp?acode=H121058
순위 :  6
상호 :  베르네닭강정
업종 :  한식
주소 :  경기 부천시 여월동
url :  /restaurant/onepage.asp?acode=H492662
순위 :  7
상호 :  봉래헌
업종 :  한식
주소 :  강서구 외발산동
url :  /restaurant/onepage.asp?acode=H119214
순위 :  8
상호 :  원조나주곰탕
업종 :  한식
주소 :  강서구 방화동
url :  /restaurant/onepage.asp?acode=H121046
순위 :  9
상호 :  빌리엔젤 목동현대41타워점
업종 :  카페/주점
주소 :  양천구 목동
url :  /restaurant/onepage.asp?acode=D103477
순위 :  10
상호 :  딘타이펑 김포점
업종 :  중식
주소 :  강서구 방화동
url :  /restaurant/onepage.asp?acode=C102870
순위 :  11
상호 :  낙원
업종 :  한식
주소 :  강서구 외발산동
url :  /restaurant/onepage.asp?acode=H119199
순위 : 

In [87]:
# 위의 데이터들을 list로 나타내는 작업


# [ {'순위' : '1', '상호' : 'XXXX', '업종' : '한식', ...},
# [ {'순위' : '2', '상호' : 'XXXX', '업종' : '한식', ...},
# [ {'순위' : '3', '상호' : 'XXXX', '업종' : '한식', ...},
# ... ]

# url에 대한 내용은 따로 별개의 list에 넣기

menuList = []
urlList = []

s = 'body > div > div.container > div.bestRest > div.ranking > div.rankingList > ul'
ulTag = soup.select_one(s)
# print(ulTag)

for li in ulTag.select('li'):
  menuList.append( {'순위' : li.select_one('.numTop, .rankNum').string,
                    '상호' : li.select_one('.restName').string,
                    '업종' : li.select_one('.listType').string,
                    '주소' : li.select_one('.listArea').string})
  urlList.append(li.select_one('.restName a')['href'])
print(menuList)
print(urlList)

AttributeError: 'NoneType' object has no attribute 'select'

In [78]:
# menuList를 가지고 순위를 index로 하는 DataFrame을 생성
menuDF = pd.DataFrame(menuList)
menuDF.set_index('순위', inplace=True)
menuDF

,상호,업종,주소
순위,,,
1,사시미야센,일식,양천구 목동
2,명가들깨칼국수,한식,양천구 신정1동
3,홍대돈부리 목동점,일식,양천구 목1동
4,팡코 목동점,베이커리,양천구 목4동
5,고성막국수,한식,강서구 방화동
6,베르네닭강정,한식,경기 부천시 여월동
7,봉래헌,한식,강서구 외발산동
8,원조나주곰탕,한식,강서구 방화동
9,빌리엔젤 목동현대41타워점,카페/주점,양천구 목동


In [82]:
# http://www.menupan.com/이 기본 주소값 뒤에 urlList에 담아놓은 주소값 붙여서 요청
# 전화번호, 상세주소 (두개 중 하나)를 가져온 후에
# 만들어놓은 DataFrame에 붙이기 ( + '주소' 열은 삭제)

baseUrl = 'http://www.menupan.com/'
telList = []
addrList = []

for u in urlList:
  response = requests.get(baseUrl + u)
  soup = BeautifulSoup(response.content, 'html5lib')
  # print(soup.select_one('.tel1').string)
  # print(soup.select_one('.add1 a').string)
  telList.append(soup.select_one('.tel1').string)
  addrList.append(soup.select_one('.add1 a').string)

menuDF['전화번호'] = telList
menuDF['상세주소'] = addrList
menuDF.drop(columns='주소', inplace=True)
menuDF

,상호,업종,전화번호,상세주소
순위,,,,
1,사시미야센,일식,(02) 2646-9808,서울 양천구 목동 406-25
2,명가들깨칼국수,한식,(02) 2653-3070,서울 양천구 신정1동 1020-15
3,홍대돈부리 목동점,일식,(02) 6123-9808,서울 양천구 목1동 917 파라곤 지하 1층 20호
4,팡코 목동점,베이커리,(02) 2652-0420,서울 양천구 목4동 754-4 삼진빌딩
5,고성막국수,한식,(02) 2665-1205,서울 강서구 방화동 323-6
6,베르네닭강정,한식,(032) 676-4692,경기 부천시 여월동 3-21
7,봉래헌,한식,(02) 2660-9020,서울 강서구 외발산동 426
8,원조나주곰탕,한식,(02) 2666-8292,서울 강서구 방화동 559-1
9,빌리엔젤 목동현대41타워점,카페/주점,(02) 2654-6575,서울 양천구 목동 917-9 목동 41타워 3층


In [83]:
# 업종이 한식인 가게의 정보만 출력
menuDF[menuDF['업종'] == '한식']

,상호,업종,전화번호,상세주소
순위,,,,
2,명가들깨칼국수,한식,(02) 2653-3070,서울 양천구 신정1동 1020-15
5,고성막국수,한식,(02) 2665-1205,서울 강서구 방화동 323-6
6,베르네닭강정,한식,(032) 676-4692,경기 부천시 여월동 3-21
7,봉래헌,한식,(02) 2660-9020,서울 강서구 외발산동 426
8,원조나주곰탕,한식,(02) 2666-8292,서울 강서구 방화동 559-1
11,낙원,한식,(02) 2660-9010,서울 강서구 외발산동 426
13,미쓰쭈,한식,(02) 2061-7423,서울 양천구 목동 917 파라곤 지하1층 22
14,김정수할머니보쌈막국수,한식,(032) 673-1150,경기 부천시 원미구 춘의동 254-9
16,공항칼국수,한식,(02) 2664-9748,서울 강서구 공항동 49-2


In [84]:
# 특정 구에 있는 가게의 정보만 출력
# Pandas에서 Series의 특정 문자열 찾기 : str.contains()

menuDF[menuDF['상세주소'].str.contains('양천구')]

,상호,업종,전화번호,상세주소
순위,,,,
1,사시미야센,일식,(02) 2646-9808,서울 양천구 목동 406-25
2,명가들깨칼국수,한식,(02) 2653-3070,서울 양천구 신정1동 1020-15
3,홍대돈부리 목동점,일식,(02) 6123-9808,서울 양천구 목1동 917 파라곤 지하 1층 20호
4,팡코 목동점,베이커리,(02) 2652-0420,서울 양천구 목4동 754-4 삼진빌딩
9,빌리엔젤 목동현대41타워점,카페/주점,(02) 2654-6575,서울 양천구 목동 917-9 목동 41타워 3층
13,미쓰쭈,한식,(02) 2061-7423,서울 양천구 목동 917 파라곤 지하1층 22
15,와플베르비에,카페/주점,(02) 2168-2730,서울 양천구 목동 917-9 현대 41타워 1층 15호
17,더더차돌감바스,한식,(02) 2654-3392,서울 양천구 목동 602-3
19,마고레스토랑,양식,(02) 2653-7115,서울 양천구 신정동 321-6 센트럴프라자 1406호
